In [1]:
import json
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# Install necessary packages using pip
!pip install selenium chromedriver_autoinstaller

# Install ChromeDriver
chromedriver_autoinstaller.install()

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
chrome_options.add_experimental_option('detach', True)
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--remote-debugging-port=9222')

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time


chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU usage
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Specify the path to the ChromeDriver executable
chrome_driver_path = '/Users/lexie_chen/Desktop/Geography Research/chromedriver-mac-x64/chromedriver'
service = Service(chrome_driver_path)

data_list = []

p_from = 1
for p in range(24):
    url = f'https://www.yelp.com/search?find_desc=Bakery&find_loc=Chester+County%2C+PA&start={p_from}'
    print(url)

    with webdriver.Chrome(service=service, options=chrome_options) as driver:
        driver.get(url)
        time.sleep(10)  # Increase sleep time to avoid bot detection

        soup = BeautifulSoup(driver.page_source, 'lxml')
        soup_list = soup.select('.y-css-1he6azc')

        if not soup_list:
            print(f"No data found on page {p}. Skipping to the next page.")
            continue

        for sl in soup_list:
            try:
                data = {}
                title_element = sl.select_one('.y-css-12ly5yx')
                data['title'] = title_element.text if title_element else 'N/A'

                rate_element = sl.select_one('.y-css-jf9frv')
                data['rate'] = rate_element.text if rate_element else 'N/A'

                review_element = sl.select_one('.y-css-1fnvi93 .y-css-wfbtsu')
                data['review'] = review_element.text if review_element else 'N/A'

                label_elements = sl.select('.y-css-1cn4gbs')
                data['label'] = '、'.join([ll.text for ll in label_elements])

                price_element = sl.select_one('.priceRange__09f24__ZgJXy.y-css-mfup3c')
                symbol = price_element.text + '·' if price_element else ''
                area_element = sl.select_one('.y-css-1lvo3zq .y-css-wfbtsu')
                data['area'] = symbol + area_element.text if area_element else 'N/A'

                time_element = sl.select_one('.y-css-blvhhm .y-css-h9c2fl')
                data['time'] = time_element.text if time_element else 'N/A'

                serve_list = []
                for fl in sl.select('.arrange__09f24__LDfbs.gutter-1__09f24__yAbCL.vertical-align-middle__09f24__zU9sE.y-css-1yi0uhn .tag__09f24__wuJ8a.y-css-79z9pg'):
                    is_yes = 'yes ' if 'M6.308 11.763a.748.' in str(fl) else 'no '
                    serve_list.append(is_yes + fl.text)
                data['service'] = '、'.join(serve_list)

                print(data)
                data_list.append(data)
            except Exception as e:
                print(f"Error on page {p}: {e}")

    if p_from == 221:
        p_from += 9
    else:
        p_from += 10

https://www.yelp.com/search?find_desc=Bakery&find_loc=Chester+County%2C+PA&start=1
{'title': 'Yori’s Church Street Bakery', 'rate': '4.3 ', 'review': '(123 reviews)', 'label': 'Bakeries、Desserts、Breakfast & Brunch', 'area': '$', 'time': 'Closes in 5 min', 'service': 'yes Delivery、yes Takeout'}
{'title': 'Malvern Buttery', 'rate': '4.3 ', 'review': '(337 reviews)', 'label': 'Cafes、Bakeries', 'area': '$$', 'time': 'Closed until 8:00 AM tomorrow', 'service': 'yes Outdoor seating、yes Takeout'}
{'title': 'Bakers of Buffington', 'rate': '3.9 ', 'review': '(80 reviews)', 'label': 'Bakeries、Patisserie/Cake Shop', 'area': '$$', 'time': 'Closed until 8:30 AM tomorrow', 'service': 'yes Delivery、yes Takeout'}
{'title': 'The Master’s Baker', 'rate': '4.0 ', 'review': '(68 reviews)', 'label': 'Desserts、Bakeries、Custom Cakes', 'area': '$$', 'time': 'Closed until 9:00 AM tomorrow', 'service': 'yes Delivery、yes Takeout、yes Curbside Pickup'}
{'title': 'Brandywine Valley Bread', 'rate': '4.1 ', 'review':

{'title': 'Sugar ‘n’ Spice Cakery', 'rate': '1.0 ', 'review': '(1 review)', 'label': 'Custom Cakes、Desserts、Bakeries', 'area': '', 'time': 'Closed until 9:00 AM tomorrow', 'service': ''}
{'title': 'Traub’s Bakery', 'rate': '3.6 ', 'review': '(123 reviews)', 'label': 'Bakeries、Custom Cakes、Donuts', 'area': '$', 'time': 'Closed until 7:00 AM tomorrow', 'service': 'yes Delivery、yes Takeout、yes Curbside Pickup'}
{'title': 'Beiler’s Bakery', 'rate': '4.0 ', 'review': '(2 reviews)', 'label': 'Bakeries、Donuts、Cupcakes', 'area': '', 'time': 'Closed until 8:00 AM tomorrow', 'service': ''}
{'title': 'Sadie’s Bake Shop', 'rate': '4.5 ', 'review': '(2 reviews)', 'label': 'Bakeries', 'area': '', 'time': 'N/A', 'service': ''}
{'title': 'Sweet Charlottes', 'rate': 'N/A', 'review': 'N/A', 'label': 'Bakeries', 'area': '', 'time': 'N/A', 'service': ''}
{'title': 'Vanessa Ross Cakes', 'rate': 'N/A', 'review': 'N/A', 'label': 'Bakeries、Cupcakes', 'area': '', 'time': 'Closed today', 'service': 'yes Deliver

{'title': 'Serpe & Sons Bakery', 'rate': '2.9 ', 'review': '(78 reviews)', 'label': 'Bakeries', 'area': '$$', 'time': 'This is a placeholder', 'service': 'yes Takeout'}
{'title': 'Corropolese Bakery & Deli', 'rate': '4.6 ', 'review': '(304 reviews)', 'label': 'Bakeries、Delis、Italian', 'area': '$', 'time': 'This is a placeholder', 'service': 'yes Outdoor seating、yes Delivery、yes Takeout'}
{'title': 'Honey Moon Bakery & Pizzeria', 'rate': '4.7 ', 'review': '(7 reviews)', 'label': 'Pizza、Bakeries、Desserts', 'area': '', 'time': 'This is a placeholder', 'service': 'yes Takeout'}
{'title': 'Bakers on Broad', 'rate': '4.8 ', 'review': '(45 reviews)', 'label': 'Bakeries', 'area': '$', 'time': 'This is a placeholder', 'service': 'yes Takeout'}
{'title': 'Achenbach’s Pastry', 'rate': '4.4 ', 'review': '(61 reviews)', 'label': 'Bakeries', 'area': '$', 'time': 'This is a placeholder', 'service': 'yes Takeout、yes Curbside Pickup'}
{'title': 'Cakes & Candies by Mary Ellen', 'rate': '3.5 ', 'review':

{'title': 'The Yummery', 'rate': '4.5 ', 'review': '(43 reviews)', 'label': 'Cupcakes、Coffee & Tea、Custom Cakes', 'area': '$$', 'time': 'Closed until 9:30 AM tomorrow', 'service': 'yes Delivery、yes Takeout'}
{'title': 'Chesapeake Bay Coffee Co', 'rate': '4.6 ', 'review': '(96 reviews)', 'label': 'Coffee & Tea', 'area': '$', 'time': 'Closed until 7:00 AM tomorrow', 'service': 'yes Outdoor seating、yes Delivery、yes Takeout'}
{'title': 'Black Buggy Baking Company', 'rate': '5.0 ', 'review': '(3 reviews)', 'label': 'Bakeries', 'area': '$', 'time': 'N/A', 'service': ''}
{'title': 'Chive Cafe', 'rate': '4.5 ', 'review': '(188 reviews)', 'label': 'Cafes、Breakfast & Brunch', 'area': '$$', 'time': 'Closed until 8:00 AM tomorrow', 'service': 'yes Outdoor seating、yes Delivery、yes Takeout'}
{'title': 'Gianino’s Pastries', 'rate': '4.5 ', 'review': '(30 reviews)', 'label': 'Bakeries', 'area': '$$', 'time': 'Closed until 8:00 AM tomorrow', 'service': 'no Curbside Pickup'}
{'title': 'Beiler’s Bakery',

{'title': 'Aunt Nannie’s Bake Shop', 'rate': '5.0 ', 'review': '(2 reviews)', 'label': 'Bakeries、Custom Cakes、Cupcakes', 'area': '', 'time': 'Closed until 8:00 AM tomorrow', 'service': ''}
{'title': 'Nothing Bundt Cakes', 'rate': '4.1 ', 'review': '(15 reviews)', 'label': 'Bakeries、Desserts、Cupcakes', 'area': '', 'time': 'Closed until 9:00 AM tomorrow', 'service': 'no Delivery、yes Takeout'}
{'title': 'Miller’s Bakery', 'rate': '4.5 ', 'review': '(2 reviews)', 'label': 'Bakeries', 'area': '', 'time': 'Closed until Noon tomorrow', 'service': ''}
{'title': 'My Old Place Bakery', 'rate': '3.9 ', 'review': '(28 reviews)', 'label': 'Donuts、Bakeries', 'area': '$', 'time': 'Closed until 7:00 AM tomorrow', 'service': 'yes Takeout'}
{'title': 'The Original Bagel', 'rate': '4.5 ', 'review': '(87 reviews)', 'label': 'Bagels、Desserts、Breakfast & Brunch', 'area': '$', 'time': 'Closed until 5:00 AM tomorrow', 'service': 'no Delivery、yes Takeout'}
{'title': 'Cakerybake', 'rate': 'N/A', 'review': 'N/A'

In [6]:
df = pd.DataFrame(data_list)
df.to_csv('all_bakeries_Chester.xlsx', index=False)
#df.to/_csv('all_bakeries_philadelphia.csv', index=False)